<a href="https://colab.research.google.com/github/dmswl0707/ResNet_Pytorch/blob/main/train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def train(epoch, model, data_loader):
    print("==========training==========")
    running_loss = 0.0
    running_acc = 0.0

    model.train()

    for e in range(epoch):      #에폭 만큼 반복
        for batch_idx, (input, target) in enumerate(data_loader):       # 배치 사이즈 만큼 반복
            global_step = e * len(data_loader)*batch_size + (batch_idx * batch_size)

            input, target = input.cuda(), target.cuda()
            input, target = Variable(input, volatile=False), Variable(target)

            optimizer.zero_grad()

            output = model(input)

            loss=F.nll_loss(output, target)
            preds = output.data.max(dim=1, keepdim=True)[1]
            acc=preds.eq(target.data.view_as(preds)).cpu().sum()
            running_loss += loss.item()
            running_acc+=acc.item()


            loss.backward()
            optimizer.step()


            if batch_idx%100==0:
                loss = running_loss/100
                accuracy = running_acc/100
                writer.add_scalar('loss', loss, global_step)        #tensorboard scalar add
                writer.add_scalar('accuracy', accuracy, global_step)

                print('Epoch: {:<4} iter: {:>4} loss {} acc {}' .format(e+1, global_step, loss, accuracy))
                running_loss = 0.0
                running_acc = 0.0

def test(model, data_loader):
    model.eval()
    volatile = True
    running_acc = 0.0

    for batch_idx, (input, target) in enumerate(data_loader):
        global_step = batch_idx * batch_size

        input, target = input.cuda(), target.cuda()
        input, target = Variable(input, volatile=True), Variable(target)

        output = model(input)
        preds = output.data.max(dim=1, keepdim=True)[1]
        acc = preds.eq(target.data.view_as(preds)).cpu().sum()
        running_acc += acc.item()



    accuracy = running_acc / len(data_loader)

    print('test acc {}'.format(accuracy))
